# Import Packages and data

In [1]:
import pandas as pd
import numpy as np
from utils.create_features import jaro, WRatio, ratio, davies, token_set_ratio, add_lat_lon_distance_features, strike_a_match, leven
from utils.features_utils import name_distance, extract_directions, is_direction_match, name_number_match,\
    is_related_cat, category_match, clean_email, sub_category_match, brand_match, house_match, email_url_match, phone_lcs,phone_category
    
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=4)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data_fold_v4.csv")

In [3]:
cols = ['country', 'placeId1', 'placeId2', 'sourceNames1', 'sourceNames2',
       'category1', 'category2', 'brands1', 'brands2', 'email1', 'email2',
       'latitude1', 'longitude1', 'latitude2', 'longitude2', 'houseNumber1',
       'houseNumber2', 'streets1', 'streets2', 'cities1', 'cities2',
       'subCategory1', 'subCategory2', 'phoneNumbers1', 'phoneNumbers2',
       'internet1', 'internet2', 'postalCode1', 'postalCode2',
       'duplicate_flag', 'similarity']

In [4]:
df_pairs = df_pairs[cols]

In [5]:
df_pairs.drop_duplicates(inplace=True)

In [6]:
df_pairs["phoneNumbers1"] = df_pairs["phoneNumbers1"].apply(eval)
df_pairs["phoneNumbers2"] = df_pairs["phoneNumbers2"].apply(eval)

# Create Features

In [7]:
import time

# get the start time
st = time.time()

In [8]:

# similarity = name_distance(list(df_pairs["sourceNames1"]),list(df_pairs["sourceNames2"]))
# df_pairs["similarity"] = similarity

In [9]:
df_pairs = add_lat_lon_distance_features(df_pairs)

In [10]:
df_pairs["direction1"] = df_pairs["sourceNames1"].apply(extract_directions)
df_pairs["direction2"] = df_pairs["sourceNames2"].apply(extract_directions)
df_pairs['Is_direction_match'] = df_pairs.apply(lambda x: is_direction_match(x.direction1, x.direction2), axis=1)


In [11]:
df_pairs["name1_number"] = df_pairs["sourceNames1"].str.extract('(\d+)')
df_pairs["name2_number"] = df_pairs["sourceNames2"].str.extract('(\d+)')
df_pairs['Is_name_number_match'] = df_pairs.apply(lambda x: name_number_match(x.name1_number, x.name2_number), axis=1)

In [12]:
df_pairs['Is_related_cat'] = df_pairs.apply(lambda x: is_related_cat(x.category1, x.category2), axis=1)
df_pairs['Is_category_match'] = df_pairs.apply(lambda x: category_match(x.category1, x.category2), axis=1)
df_pairs['Is_subcategory_match'] = df_pairs.apply(lambda x: sub_category_match(x.subCategory1, x.subCategory2), axis=1)

In [13]:
df_pairs['Is_brand_match'] = df_pairs.apply(lambda x: brand_match(x.brands1, x.brands2), axis=1)
df_pairs['Is_house_match'] = df_pairs.apply(lambda x: house_match(x.houseNumber1, x.houseNumber2), axis=1)
df_pairs['is_phone_match'] = df_pairs.apply(lambda x: phone_category(x.phoneNumbers1, x.phoneNumbers2), axis=1)

In [14]:
df_pairs['Is_email_match'] = df_pairs.apply(lambda x: email_url_match(x.email1, x.email2), axis=1)
df_pairs['Is_url_match'] = df_pairs.apply(lambda x: email_url_match(x.internet1, x.internet2), axis=1)


In [15]:
df_pairs['name_davies'] = df_pairs.apply(lambda x: davies(x.sourceNames1, x.sourceNames2), axis=1)
df_pairs['name_leven'] = df_pairs.apply(lambda x: leven(x.sourceNames1, x.sourceNames2), axis=1)
df_pairs['name_dice'] = df_pairs.apply(lambda x: strike_a_match(x.sourceNames1, x.sourceNames2), axis=1)
df_pairs['name_jaro'] = df_pairs.apply(lambda x: jaro(x.sourceNames1, x.sourceNames2), axis=1)
df_pairs['name_set_ratio'] = df_pairs.apply(lambda x: token_set_ratio(x.sourceNames1, x.sourceNames2), axis=1)

In [16]:
df_pairs['street_davies'] = df_pairs.apply(lambda x: davies(x.streets1, x.streets2), axis=1)
df_pairs['street_leven'] = df_pairs.apply(lambda x: leven(x.streets1, x.streets2), axis=1)
df_pairs['street_jaro'] = df_pairs.apply(lambda x: jaro(x.streets1, x.streets2), axis=1)

In [17]:
df_pairs['email_davies'] = df_pairs.apply(lambda x: davies(x.email1, x.email2), axis=1)
df_pairs['email_leven'] = df_pairs.apply(lambda x: leven(x.email1, x.email2), axis=1)
df_pairs['email_jaro'] = df_pairs.apply(lambda x: jaro(x.email1, x.email2), axis=1)

In [18]:
df_pairs['url_davies'] = df_pairs.apply(lambda x: davies(x.internet1, x.internet2), axis=1)
df_pairs['url_leven'] = df_pairs.apply(lambda x: leven(x.internet1, x.internet2), axis=1)
df_pairs['url_jaro'] = df_pairs.apply(lambda x: jaro(x.internet1, x.internet2), axis=1)

In [19]:
df_pairs['brands_davies'] = df_pairs.apply(lambda x: davies(x.brands1, x.brands2), axis=1)
df_pairs['brand_leven'] = df_pairs.apply(lambda x: leven(x.brands1, x.brands2), axis=1)
df_pairs['brand_jaro'] = df_pairs.apply(lambda x: jaro(x.brands1, x.brands2), axis=1)

In [20]:
df_pairs['phone_lcs'] = df_pairs.apply(lambda x: phone_lcs(x.phoneNumbers1, x.phoneNumbers2), axis=1)
df_pairs['subcat_WRatio'] = df_pairs.apply(lambda x: WRatio(x.subCategory1, x.subCategory2), axis=1)
df_pairs['subcat_ratio'] = df_pairs.apply(lambda x: ratio(x.subCategory1, x.subCategory2), axis=1)
df_pairs['subcat_token_set_ratio'] = df_pairs.apply(lambda x: token_set_ratio(x.subCategory1, x.subCategory2), axis=1)

In [21]:
cat_columns = [
    'Is_direction_match', 'Is_house_match', 'Is_category_match',
    'Is_subcategory_match', 'Is_brand_match', 'Is_related_cat',
    'Is_name_number_match', 'is_phone_match', 'Is_email_match', 'Is_url_match'
]


In [22]:
df_pairs = pd.get_dummies(df_pairs, columns = cat_columns)

In [23]:
# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 764.8266484737396 seconds


In [24]:
df_pairs.columns

Index(['country', 'placeId1', 'placeId2', 'sourceNames1', 'sourceNames2',
       'category1', 'category2', 'brands1', 'brands2', 'email1', 'email2',
       'latitude1', 'longitude1', 'latitude2', 'longitude2', 'houseNumber1',
       'houseNumber2', 'streets1', 'streets2', 'cities1', 'cities2',
       'subCategory1', 'subCategory2', 'phoneNumbers1', 'phoneNumbers2',
       'internet1', 'internet2', 'postalCode1', 'postalCode2',
       'duplicate_flag', 'similarity', 'haversine', 'direction1', 'direction2',
       'name1_number', 'name2_number', 'name_davies', 'name_leven',
       'name_dice', 'name_jaro', 'name_set_ratio', 'street_davies',
       'street_leven', 'street_jaro', 'email_davies', 'email_leven',
       'email_jaro', 'url_davies', 'url_leven', 'url_jaro', 'brands_davies',
       'brand_leven', 'brand_jaro', 'phone_lcs', 'subcat_WRatio',
       'subcat_ratio', 'subcat_token_set_ratio', 'Is_direction_match_0',
       'Is_direction_match_1', 'Is_direction_match_2', 'Is_house_mat

In [26]:
cols = [
    'country', 'placeId1', 'placeId2', 'sourceNames1', 'sourceNames2',
    'category1', 'category2', 'brands1', 'brands2', 'email1', 'email2',
    'latitude1', 'longitude1', 'latitude2', 'longitude2', 'houseNumber1',
    'houseNumber2', 'streets1', 'streets2', 'cities1', 'cities2',
    'subCategory1', 'subCategory2', 'phoneNumbers1', 'phoneNumbers2',
    'internet1', 'internet2', 'postalCode1', 'postalCode2', 'duplicate_flag',
    'similarity', 'haversine','name_davies', 'name_leven', 'name_dice', 'name_jaro',
    'name_set_ratio', 'street_davies', 'street_leven', 'street_jaro',
    'email_davies', 'email_leven', 'email_jaro', 'url_davies', 'url_leven',
    'url_jaro', 'brands_davies', 'brand_leven', 'brand_jaro', 'phone_lcs',
    'subcat_WRatio', 'subcat_ratio', 'subcat_token_set_ratio',
    'Is_direction_match_0', 'Is_direction_match_1', 'Is_direction_match_2',
    'Is_house_match_0', 'Is_house_match_1', 'Is_house_match_2',
    'Is_category_match_0', 'Is_category_match_1', 'Is_subcategory_match_0',
    'Is_subcategory_match_1', 'Is_subcategory_match_2', 'Is_brand_match_0',
    'Is_brand_match_1', 'Is_brand_match_2', 'Is_brand_match_3',
    'Is_related_cat_0', 'Is_related_cat_1', 'Is_name_number_match_0',
    'Is_name_number_match_1', 'Is_name_number_match_2',
    'Is_name_number_match_3', 'is_phone_match_1', 'is_phone_match_2',
    'is_phone_match_3', 'is_phone_match_4', 'Is_email_match_0',
    'Is_email_match_1', 'Is_email_match_2', 'Is_url_match_0', 'Is_url_match_1',
    'Is_url_match_2'
]


In [28]:
df_pairs[cols].to_csv("/workspace/Training_data/training/Full_training_data_fold_v4.csv",index=None)